The cell below will make the notebook cells wider than default...

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

RESTART kernel prior to running after any changes to config files...

### import configuration and functions modules

In [ ]:
%%time
import functions as f
import list_builder as lb
import pandas as pd

### build program files
dataframe pickle files: master, fur, sg(special group), active_each_month, order proposals, month percentage

dictionary pickle files: settings_dict, color_dict

pay data pickle files: pay_table_basic, pay_table_enhanced

pay data spreadsheet workbook: pay_table_data.xlsx (in reports folder)

#### note: add case study name after script name

In [ ]:
%%time
%run build_program_files sample3

### skeleton

In [ ]:
%%time
%run make_skeleton

### standalone
standalone must be rerun after job level model change...

In [ ]:
%%time
%run standalone prex

### calculate for each list order
(including list conditions, job count changes, and possible furlough recall schedule)

The cells below run the compute_measures script with proposals p1, p2, and p3.  All scenarios include a "prex" or pre-existing job assignment condition.  The first case includes a ratio condition and the second case includes a count-ratio condition.

In [ ]:
%%time
%run compute_measures p1 prex ratio

In [ ]:
%%time
%run compute_measures p2 prex count

In [ ]:
%%time
%run compute_measures p3 prex

### merge an ordered list with the master list

In [ ]:
%%time
%run join_inactives p3 ffill

In [ ]:
%%time
f.print_config_selections()

### cells below for demonstration only

### build a 'hybrid' list example:
(stored as 'dill/hybrid.pkl')

In [ ]:
%%time
# this will build a "hybrid" list with equal weighting given to longevity and standalone job position
master = lb.prepare_master_list()
lb.build_list(master, ['ldate', 'jobp'], [.5, .5])

In [ ]:
%%time
# generate dataset for the hybrid list
%run compute_measures hybrid prex

### sample of files created

In [ ]:
%%time
# print only the first 5 lines of each file
case_study = pd.read_pickle('dill/case_dill.pkl').case.value
file_names = ['master', 'skeleton', 'standalone', 'p_p1', 'pay_table_enhanced', 'pay_table_basic', 'ds_p1', 'p_hybrid', case_study + '_final']
for file in file_names:
    try:
        df = pd.read_pickle('dill/' + file + '.pkl')
        print('< ' + file.upper() + ' >', '\n', 'row count: ' + str(len(df)),
            '\n', 'column count: '+ str(len(df.columns)), '\n',
            df.columns, '\n',
            '\n', df.head().to_string(line_width=120), '\n\n', '-' * 80, '\n')
    except:
        pass